# 🎵 Spotify Artist Catalog & Unclaimed Musical Works Cross-Reference

## Project Overview

This project demonstrates working with APIs, large datasets, and memory-efficient processing to solve a real-world music data challenge.

## Goals

1. **Retrieve Spotify Artist Catalog:**  
   - Use the Spotify API to fetch a complete catalog of a chosen artist.  
   - Include all albums, singles, EPs, and tracks.

2. **Export Catalog with Details:**  
   - Track name, album name, release date, and ISRC codes.  
   - Handle API rate limits safely to avoid request failures.

3. **Cross-Reference with Large Dataset:**  
   - Compare artist ISRCs against the unclaimed works dataset (`unclaimedmusicalworkrightshares.tsv`).  
   - Use memory-efficient chunked processing to handle 7GB TSV files.

4. **Generate Organized Excel Report:**  
   - Sheet 1: Full Artist Catalog.  
   - Sheet 2: Matches found in unclaimed dataset.  
   - Sheet 3: Notes and process details.

## Highlights

- **API Integration:** Spotify Web API used for reliable data retrieval.  
- **Memory-Efficient Processing:** Large TSV processed in chunks to avoid memory overflow.  
- **Data Matching:** ISRC codes cleaned and standardized to ensure accurate cross-referencing.  
- **Output:** Clean, structured Excel workbook summarizing artist tracks and unclaimed works matches.


In [1]:
import pandas as pd
file_path = "C:/Users/deepa/Placement_Assesment/data/assessment.tsv"
df0 = pd.read_csv(file_path, sep='\t', nrows=0)
df0.columns.tolist()

['#UnclaimedMusicalWorkRightShareRecordId',
 'ResourceRecordId',
 'MusicalWorkRecordId',
 'ISRC',
 'DspResourceId',
 'ResourceTitle',
 'ResourceSubTitle',
 'AlternativeResourceTitle',
 'DisplayArtistName',
 'DisplayArtistISNI',
 'Duration',
 'UnclaimedRightSharePercentage',
 'PercentileForPrioritisation']

In [2]:
sample = pd.read_csv(file_path, sep='\t', nrows=5)
sample

,#UnclaimedMusicalWorkRightShareRecordId,ResourceRecordId,MusicalWorkRecordId,ISRC,DspResourceId,ResourceTitle,ResourceSubTitle,AlternativeResourceTitle,DisplayArtistName,DisplayArtistISNI,Duration,UnclaimedRightSharePercentage,PercentileForPrioritisation
0,10257123,45_Tra.56132724,W0963A,USCM50300149,melodyvr::Tra.56132724,WE CAN SMOKE,NaN,NaN,BIG TYMERS,NaN,348,12.5,NaN
1,10257141,45_Tra.356056553,HA3LHN,GBKQU1876813,melodyvr::Tra.356056553,HIGH ROLLERS,NaN,NaN,ALCOSTA BLVD,NaN,244,50.0,NaN
2,10257164,45_Tra.224697054,AV957Y,BEK011600042,melodyvr::Tra.224697054,ARCADE,NaN,NaN,DIMITRI VEGAS & LIKE MIKE,NaN,285,25.0,NaN
3,10257193,45_Tra.644624494,TVCOAI,US2642149111,melodyvr::Tra.644624494,THE SCREAMS (2022 MIX),NaN,NaN,INTEGRITY,NaN,175,50.0,NaN
4,10257198,45_Tra.790295191,PI72QT,USLD91751743,melodyvr::Tra.790295191,PAY ATTENTION (FEAT. D- STYLES),NaN,NaN,LEE SCOTT,NaN,135,50.0,NaN


In [3]:
# counting total rows in the dataset
import csv
with open(file_path, 'r', encoding='utf-8') as f:
    row_count = sum(1 for _ in f) - 1  # minus header
print("Total rows:", row_count)

Total rows: 59989690


In [4]:
#checking some random rows
sample_rows = pd.read_csv(file_path, sep='\t', skiprows=lambda i: i>0 and i%100000!=0, nrows=200, dtype=str)
sample_rows.head()

,#UnclaimedMusicalWorkRightShareRecordId,ResourceRecordId,MusicalWorkRecordId,ISRC,DspResourceId,ResourceTitle,ResourceSubTitle,AlternativeResourceTitle,DisplayArtistName,DisplayArtistISNI,Duration,UnclaimedRightSharePercentage,PercentileForPrioritisation
0,50201102,2_spotify:track:2jmaDR4LMXvOSVwkFt9YJj,GV8JYQ,DEW871503411,spotify::spotify:track:2jmaDR4LMXvOSVwkFt9YJj,GOOD TIMES,NaN,NaN,NEV SCOTT,NaN,403,10.0,NaN
1,10118026,22_1790766085,BD6XKY,USUYG1580417,applemusic::1790766085,AMMO,NaN,NaN,SHORDIE SHORDIE,NaN,184,25.0,NaN
2,50184491,2_spotify:track:2IiCxBFyfsJnjqqFW6zfIN,M1841L,USTV10300135,spotify::spotify:track:2IiCxBFyfsJnjqqFW6zfIN,MADE TO LIE,NaN,NaN,DEFAULT,NaN,195,20.0,NaN
3,10036547,15_dQKelpy1-pY,L8430Y,USAR30100183,youtube::dQKelpy1-pY,LIVE FOR YOU,NaN,NaN,BIG TENT REVIVAL,NaN,397,50.0,NaN
4,14289810,22_52456,S28348,USPR39882290,applemusic::52456,6/8 - 7/8,NaN,NaN,THE HEADHUNTERS,NaN,416,60.0,NaN


## Sample Rows from Large TSV

**Purpose:** Preview dataset without loading full 7GB file.

**Method:**  
- Read file in chunks (`chunksize=200k`).  
- Randomly sample 5 rows per chunk.  
- Combine samples (~50 rows total) for inspection.

**Benefit:** Efficient memory usage, quick dataset overview.

In [5]:
samples = []
chunksize = 200000
for chunk in pd.read_csv(file_path, sep='\t', chunksize=chunksize, dtype=str):
    samples.append(chunk.sample(n=min(5, len(chunk)), random_state=1))
    if len(samples) >= 10:  # ~50 rows total
        break
sample_df = pd.concat(samples, ignore_index=True)
sample_df.head()

,#UnclaimedMusicalWorkRightShareRecordId,ResourceRecordId,MusicalWorkRecordId,ISRC,DspResourceId,ResourceTitle,ResourceSubTitle,AlternativeResourceTitle,DisplayArtistName,DisplayArtistISNI,Duration,UnclaimedRightSharePercentage,PercentileForPrioritisation
0,4545468,25_38297084,Z5926M,TCACK1568791,iheartradio::38297084,MI CHICA ESTA LOCA (FEAT. PITBULL),NaN,NaN,MATT HUNTER,NaN,40,14.31,NaN
1,9996431,15_A177162882979847,S8055I,DEA450303930,youtube::A177162882979847,SAILING AWAY,NaN,NaN,AXEL RUDI PELL,NaN,351,40.0,NaN
2,10421181,13_113785362,SVDN8N,DEU601506025,deezer::113785362,SHINING STAR,NaN,NaN,RHAPSODY OF FIRE,NaN,279,12.93,NaN
3,14412903,30_263306959,W4715M,HKUM71000579,tidal::263306959,完美孤独,NaN,NaN,莫文蔚,NaN,291,50.0,NaN
4,14370340,27_94603049,BC3QF0,QZDA52072030,pandora::94603049,BABY DRIVER (FEAT. A$AP ANT),NaN,NaN,A$AP TWELVYY,NaN,125,75.0,NaN


In [6]:
# Getting the unique ISRC from our dataset
import duckdb

con = duckdb.connect()
query = """
SELECT DISTINCT NULLIF(trim(upper(regexp_replace(ISRC, '-', ''))), '') AS isrc
FROM read_csv_auto('C:/Users/deepa/Placement_Assesment/data/assessment.tsv', delim='\t', header=True)
WHERE ISRC IS NOT NULL
"""
unique_isrc_df = con.execute(query).df()
unique_isrc_df.to_csv('unique_isrcs_from_tsv.csv', index=False)
print("Unique ISRC count:", len(unique_isrc_df))


Unique ISRC count: 10487372


In [7]:
import os, time
import pandas as pd
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.exceptions import SpotifyException
from dotenv import load_dotenv
load_dotenv()

True

### Initialize Spotify API
- Load environment variables for `SPOTIPY_CLIENT_ID` and `SPOTIPY_CLIENT_SECRET`.
- Initialize the Spotify API client.
- Define a safe Spotify call function to handle rate limits automatically.

In [8]:
# Initialize Spotify API
sp = Spotify(
    auth_manager=SpotifyClientCredentials(
        client_id=os.getenv('SPOTIPY_CLIENT_ID'),
        client_secret=os.getenv('SPOTIPY_CLIENT_SECRET')
    ),
    requests_timeout=30
)

In [9]:
# Safe Spotify call with automatic rate-limit handling
def safe_spotify_call(func, *args, **kwargs):
    while True:
        try:
            return func(*args, **kwargs)
        except SpotifyException as e:
            if e.http_status == 429:  # rate limit
                retry_after = int(e.headers.get('Retry-After', '5'))
                print(f"Rate limit hit. Waiting {retry_after} seconds...")
                time.sleep(retry_after + 1)
            else:
                raise e
        except Exception as e:
            print("Spotify call error:", e)
            time.sleep(5)

## **Fetch Artist Catalog**

- Retrieve artist ID using Spotify search API.
- Fetch all albums and singles of the artist.
- Retrieve tracks from each album.
- Collect ISRC codes for all tracks.
- Store results in a DataFrame with relevant columns.

In [10]:
# Functions using safe_spotify_call
def get_artist_id(name):
    res = safe_spotify_call(sp.search, q=f'artist:{name}', type='artist', limit=1)
    items = res.get('artists', {}).get('items', [])
    if not items:
        raise ValueError("Artist not found")
    return items[0]['id']

def fetch_all_albums(artist_id):
    albums = []
    results = safe_spotify_call(sp.artist_albums, artist_id, album_type='album,single,compilation,appears_on', limit=50)
    albums.extend(results['items'])
    while results.get('next'):
        results = safe_spotify_call(sp.next, results)
        albums.extend(results['items'])
    albums = list({a['id']: a for a in albums}.values())
    print(f"Total albums fetched: {len(albums)}")
    return albums

def fetch_tracks_for_album(album, album_index=None, total_albums=None):
    results = safe_spotify_call(sp.album_tracks, album['id'], limit=50)
    tracks = results['items']
    while results.get('next'):
        results = safe_spotify_call(sp.next, results)
        tracks.extend(results['items'])
    if album_index is not None:
        print(f"Fetched {len(tracks)} tracks from album {album_index+1}/{total_albums}: {album['name']}")
    return [{'track_id': t['id'], 'track_name': t['name'], 
             'album_id': album['id'], 'album_name': album['name'], 
             'release_date': album.get('release_date')} for t in tracks]

def fetch_isrc_for_tracks(tracks_df):
    ids = tracks_df['track_id'].dropna().unique().tolist()
    isrc_map = {}
    total_batches = (len(ids) + 49) // 50
    for i in range(0, len(ids), 50):
        batch_num = (i // 50) + 1
        batch = ids[i:i+50]
        recs = safe_spotify_call(sp.tracks, batch)['tracks']
        for r in recs:
            if r is None: continue
            isrc_map[r['id']] = r.get('external_ids', {}).get('isrc')
        print(f"Processed batch {batch_num}/{total_batches} of track ISRCs")
        time.sleep(0.1)
    tracks_df['isrc'] = tracks_df['track_id'].map(isrc_map)
    tracks_df['isrc'] = tracks_df['isrc'].astype(str).str.upper().str.strip().str.replace('-', '', regex=False)
    tracks_df.loc[tracks_df['isrc'] == 'None', 'isrc'] = None
    return tracks_df.drop_duplicates(subset=['isrc', 'track_id'])

In [11]:
# Build full catalog
def build_artist_catalog(artist_name):
    artist_id = get_artist_id(artist_name)
    albums = fetch_all_albums(artist_id)
    all_tracks = []
    total_albums = len(albums)
    for idx, album in enumerate(albums):
        all_tracks.extend(fetch_tracks_for_album(album, album_index=idx, total_albums=total_albums))
    df = pd.DataFrame(all_tracks).drop_duplicates(subset=['track_id'])
    df = fetch_isrc_for_tracks(df)
    return df[['track_name', 'album_name', 'release_date', 'track_id', 'isrc']]

In [12]:
# Example usage 
artist_name = "Coldplay"
artist_df = build_artist_catalog(artist_name)
artist_df.to_csv('artist_catalog_with_isrcs.csv', index=False)
print("Tracks fetched:", len(artist_df))

Total albums fetched: 1684
Fetched 10 tracks from album 1/1684: Moon Music
Fetched 20 tracks from album 2/1684: Moon Music (Full Moon Edition)
Fetched 12 tracks from album 3/1684: Music Of The Spheres
Fetched 16 tracks from album 4/1684: Everyday Life
Fetched 24 tracks from album 5/1684: Live in Buenos Aires
Fetched 11 tracks from album 6/1684: A Head Full of Dreams
Fetched 9 tracks from album 7/1684: Ghost Stories Live 2014
Fetched 9 tracks from album 8/1684: Ghost Stories
Fetched 15 tracks from album 9/1684: Live 2012
Fetched 14 tracks from album 10/1684: Mylo Xyloto
Fetched 9 tracks from album 11/1684: LeftRightLeftRightLeft (Live)
Fetched 18 tracks from album 12/1684: Viva La Vida (Prospekt's March Edition)
Fetched 10 tracks from album 13/1684: Viva La Vida or Death and All His Friends
Fetched 13 tracks from album 14/1684: X&Y
Fetched 12 tracks from album 15/1684: Live 2003
Fetched 11 tracks from album 16/1684: A Rush of Blood to the Head
Fetched 10 tracks from album 17/1684: Parac

## **Cross-Reference with Large TSV Dataset**

- `unclaimedmusicalworkrightshares.tsv` is too large to load entirely (7GB).  
- Process the dataset in chunks using `chunksize`.
- Standardize ISRC codes for accurate matching.
- Filter matches with the artist catalog ISRCs.
- Combine matched chunks into a final DataFrame.

In [13]:
import pandas as pd

def find_matches_in_large_tsv(tsv_path, artist_df, chunksize=100000):
    """
    Process large TSV in chunks, clean ISRC, and merge with artist catalog.
    Returns only the matching rows.
    """
    matches = []
    artist_isrcs = set(
        artist_df["isrc"].dropna().str.upper().str.strip().str.replace("-", "", regex=False).unique()
    )
    print(f"Searching {len(artist_isrcs)} ISRCs in dataset...")

    for chunk_num, chunk in enumerate(pd.read_csv(tsv_path, sep="\t", dtype=str, chunksize=chunksize, low_memory=False)):
        if "ISRC" not in chunk.columns:
            continue
        chunk["ISRC"] = chunk["ISRC"].astype(str).str.upper().str.strip().str.replace("-", "", regex=False)

        # Filter matches
        chunk_matches = chunk[chunk["ISRC"].isin(artist_isrcs)]
        if not chunk_matches.empty:
            print(f"✅ Found {len(chunk_matches)} matches in chunk {chunk_num}")
            matches.append(chunk_matches)

    if matches:
        return pd.concat(matches, ignore_index=True)
    else:
        return pd.DataFrame(columns=["ISRC"])

In [14]:
# load artist catalog created earlier
# (This should have columns: track_name, album_name, release_date, track_id, isrc)
artist_df = pd.read_csv("artist_catalog_with_isrcs.csv")

print("Tracks fetched from Spotify catalog:", len(artist_df))
artist_df.head()

Tracks fetched from Spotify catalog: 69488


,track_name,album_name,release_date,track_id,isrc
0,MOON MUSiC,Moon Music,2024-10-04,41FNZsY7w7KaTQ2bjxdR6w,GBAYE2400882
1,feelslikeimfallinginlove,Moon Music,2024-10-04,49S3znqBAQyPyMpEuKeyJ6,GBAYE2400891
2,WE PRAY,Moon Music,2024-10-04,7xrEnNo99wrmIs8ZK3RZMK,GBAYE2400890
3,JUPiTER,Moon Music,2024-10-04,3EbRbM7qyJq9qjRqDIwBTO,GBAYE2400883
4,GOOD FEELiNGS,Moon Music,2024-10-04,65wzicJctsW9GwnTnLWxQO,GBAYE2400884


## **Export Results to Excel**

- Save all results in a single Excel file `final_artist_unclaimed.xlsx`.
- Sheet 1: Artist Catalog with ISRCs.
- Sheet 2: Matches (tracks found in unclaimed works).
- Sheet 3: Notes & Observations about the workflow.
- Memory-efficient and fully automated.

In [15]:
# Cross-reference artist catalog with large TSV
matches_df = find_matches_in_large_tsv("data/assessment.tsv", artist_df)

print("Tracks found in unclaimed dataset:", len(matches_df))

# Save to Excel
with pd.ExcelWriter("final_artist_unclaimed.xlsx", engine="openpyxl") as writer:
    artist_df.to_excel(writer, sheet_name="Artist Catalog", index=False)
    matches_df.to_excel(writer, sheet_name="Matches", index=False)

    notes_df = pd.DataFrame([
        ["Notes", "Used Spotify API to fetch full artist catalog."],
        ["Process", "Unclaimed dataset processed in chunks of 100k rows (7GB file)."],
        ["Matches Found", str(len(matches_df))],
    ], columns=["Key", "Value"])
    notes_df.to_excel(writer, sheet_name="Notes", index=False)

print("✅ Excel file created: final_artist_unclaimed.xlsx")

Searching 11510 ISRCs in dataset...
✅ Found 901 matches in chunk 0
✅ Found 888 matches in chunk 1
✅ Found 831 matches in chunk 2
✅ Found 752 matches in chunk 3
✅ Found 755 matches in chunk 4
✅ Found 862 matches in chunk 5
✅ Found 751 matches in chunk 6
✅ Found 854 matches in chunk 7
✅ Found 889 matches in chunk 8
✅ Found 788 matches in chunk 9
✅ Found 896 matches in chunk 10
✅ Found 792 matches in chunk 11
✅ Found 804 matches in chunk 12
✅ Found 827 matches in chunk 13
✅ Found 932 matches in chunk 14
✅ Found 916 matches in chunk 15
✅ Found 781 matches in chunk 16
✅ Found 898 matches in chunk 17
✅ Found 858 matches in chunk 18
✅ Found 928 matches in chunk 19
✅ Found 814 matches in chunk 20
✅ Found 885 matches in chunk 21
✅ Found 685 matches in chunk 22
✅ Found 876 matches in chunk 23
✅ Found 793 matches in chunk 24
✅ Found 840 matches in chunk 25
✅ Found 745 matches in chunk 26
✅ Found 797 matches in chunk 27
✅ Found 738 matches in chunk 28
✅ Found 796 matches in chunk 29
✅ Found 807 ma

## **Project Summary**

This project successfully demonstrates:

- Integration with the Spotify API to fetch complete artist catalogs.
- Efficient handling of large datasets (7GB TSV) using chunk processing.
- Accurate ISRC-based matching between artist tracks and unclaimed musical works.
- Production of a clean, organized Excel workbook with all relevant outputs.
- Scalable and reusable workflow suitable for any artist and large datasets.

<div style="text-align: center; font-size: 48px;">
    🙏 THANK YOU! 🙏
</div>
